task name: Implement SdcaMaximumEntropyWithOptions
task description: Implement the SdcaMaximumEntropyWithOptions example using the provided C# code and save the output to a file.

In [ ]:
#r "nuget:Microsoft.ML,1.5.5"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
// Example with label and 20 feature values. A data set is a collection of such examples.
private class DataPoint
{
    public uint Label { get; set; }
    [VectorType(20)]
    public float[] Features { get; set; }
}

// GenerateRandomDataPoints method implementation
private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count,
    int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)(random.NextDouble() - 0.5);
    for (int i = 0; i < count; i++)
    {
        var label = random.Next(1, 4);
        yield return new DataPoint
        {
            Label = (uint)label,
            Features = Enumerable.Repeat(label, 20)
                .Select(x => randomFloat() + label * 0.2f).ToArray()
        };
    }
}

var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);
trainingData = mlContext.Data.Cache(trainingData);

In [ ]:
var options = new SdcaMaximumEntropyMulticlassTrainer.Options
{
    ConvergenceTolerance = 0.05f,
    MaximumNumberOfIterations = 30,
};

var pipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy(options));

In [ ]:
var model = pipeline.Fit(trainingData);

In [ ]:
// Class used to capture predictions.
private class Prediction
{
    public uint Label { get; set; }
    public uint PredictedLabel { get; set; }
}

var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);
var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();

In [ ]:
// Pretty-print MulticlassClassificationMetrics objects.
public static void PrintMetrics(MulticlassClassificationMetrics metrics)
{
    Console.WriteLine($"Micro Accuracy: {metrics.MicroAccuracy:F2}");
    Console.WriteLine($"Macro Accuracy: {metrics.MacroAccuracy:F2}");
    Console.WriteLine($"Log Loss: {metrics.LogLoss:F2}");
    Console.WriteLine($"Log Loss Reduction: {metrics.LogLossReduction:F2}\n");

    Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());
}

foreach (var p in predictions.Take(5))
{
    Console.WriteLine($"Label: {p.Label}, Prediction: {p.PredictedLabel}");
}

var metrics = mlContext.MulticlassClassification.Evaluate(transformedTestData);
PrintMetrics(metrics);